In [ ]:
"This Jupyter Notebook is a copy Databricks notebook and was used in the Streaming Data processing part of this project"

In [ ]:
from pyspark.sql.functions import *
import urllib

In [ ]:
"""
    Why was this code used?
    
    This code was used to define the path to the delta tables and read the delta tables into the Spark dataframe.
"""
delta_table_path = "dbfs:/user/hive/warehouse/authentication_credentials"
aws_keys_df = spark.read.format("delta").load(delta_table_path)

In [ ]:
"""
    Why was this code used?
    
    This code was used to define the path to the delta tables and read the delta tables into the Spark dataframe.
"""
ACCESS_KEY = aws_keys_df.select('Access key ID').collect()[0]['Access key ID']
SECRET_KEY = aws_keys_df.select('Secret access key').collect()[0]['Secret access key']
ENCODED_SECRET_KEY = urllib.parse.quote(string=SECRET_KEY, safe="")

In [ ]:
%fs ls databricks-datasets/structured-streaming/events/file-0.json

In [ ]:
%sql

SET spark.databricks.delta.formatCheck.enabled=false

In [ ]:
"""
    Why was this code used?
    
    This code was used to Define the path to the JSON file and Read the content of the JSON file into a DataFrame
"""

jsonFilePath = "dbfs:/databricks-datasets/structured-streaming/events/file-0.json"
jsonContentDF = spark.read.json(jsonFilePath)
display(jsonContentDF)

In [ ]:
"""
Why was this code used?

This code was used to Define the schema to speed up processing for the geo data. 
It also identifies all the relevent columns in the table.
This process was esential as it ensured the data was in the correct format
The same process is used for the user and pin data.
    
"""
from pyspark.sql.types import StructType, StructField, StringType, TimestampType,MapType
from pyspark.sql.functions import from_json

inputPath = "/databricks-datasets/structured-streaming/events/"
df_stream_geo = spark \
.readStream \
.format('kinesis') \
.option('streamName','streaming-0abf7f0cd605-geo') \
.option('initialPosition','earliest') \
.option('region','us-east-1') \
.option('awsAccessKey', ACCESS_KEY) \
.option('awsSecretKey', SECRET_KEY) \
.load()

df_stream_geo = df_stream_geo.selectExpr("CAST(data as STRING)")
jsonSchema = StructType([
        StructField("ind",StringType(), True),
        StructField("timestamp", StringType(), True),
        StructField("latitude", StringType(), True),
        StructField("longitude", StringType(), True),
        StructField("country", StringType(), True)
                   ])

df_stream_geo = df_stream_geo.withColumn('jsonData',from_json(col('data'),jsonSchema)).select('jsonData.*')


display(df_stream_geo)


In [ ]:
df_stream_user = spark \
.readStream \
.format('kinesis') \
.option('streamName','streaming-0abf7f0cd605-user') \
.option('initialPosition','earliest') \
.option('region','us-east-1') \
.option('awsAccessKey', ACCESS_KEY) \
.option('awsSecretKey', SECRET_KEY) \
.load()

df_stream_user = df_stream_user.selectExpr("CAST(data as STRING)")
jsonSchema = StructType([
        StructField("ind",StringType(), True),
        StructField("first_name", StringType(), True),
        StructField("last_name", StringType(), True),
        StructField("age", StringType(), True),
        StructField("date_joined", StringType(), True)

                   ])

df_stream_user = df_stream_user.withColumn('jsonData',from_json(col('data'),jsonSchema)).select('jsonData.*')


display(df_stream_user)

In [ ]:


from pyspark.sql.types import StructType, StructField, StringType, TimestampType,MapType
from pyspark.sql.functions import from_json

inputPath = "/databricks-datasets/structured-streaming/events/"

df_stream_pin = spark \
.readStream \
.format('kinesis') \
.option('streamName','streaming-0abf7f0cd605-pin') \
.option('initialPosition','earliest') \
.option('region','us-east-1') \
.option('awsAccessKey', ACCESS_KEY) \
.option('awsSecretKey', SECRET_KEY) \
.load()
df_stream_pin = df_stream_pin.selectExpr("CAST(data AS STRING)")

jsonSchema = StructType([
        StructField("index", StringType(), True),
        StructField("unique_id", StringType(), True),
        StructField("title", StringType(), True),
        StructField("description", StringType(), True),
        StructField("poster_name", StringType(), True),
        StructField("follower_count", StringType(), True),
        StructField("tag_list", StringType(), True),
        StructField("is_image_or_video", StringType(), True),
        StructField("image_src", StringType(), True),
        StructField("downloaded", StringType(), True),
        StructField("save_location", StringType(), True),
        StructField("category", StringType(), True),
                   ])

df_stream_pin = df_stream_pin.withColumn('jsonData',from_json(col('data'),jsonSchema)).select('jsonData.*')

display(df_stream_pin)

index,unique_id,title,description,poster_name,follower_count,tag_list,is_image_or_video,image_src,downloaded,save_location,category
7528,fbe53c66-3442-4773-b19e-d3ec6f54dddf,No Title Data Available,No description available Story format,User Info Error,User Info Error,"N,o, ,T,a,g,s, ,A,v,a,i,l,a,b,l,e",multi-video(story page format),Image src error.,0,Local save in /data/mens-fashion,mens-fashion
2863,9bf39437-42a6-4f02-99a0-9a0383d8cd70,25 Super Fun Summer Crafts for Kids - Of Life and Lisa,Keep the kids busy this summer with these easy diy crafts and projects. Creative and…,Of Life & Lisa | Lifestyle Blog,124k,"Summer Crafts For Kids,Fun Crafts For Kids,Summer Kids,Toddler Crafts,Crafts To Do,Diy For Kids,Summer Snow,Diys For Summer,Craft Ideas For Girls",image,https://i.pinimg.com/originals/b3/bc/e2/b3bce2964e8c8975387b39660eed5f16.jpg,1,Local save in /data/diy-and-crafts,diy-and-crafts
5730,1e1f0c8b-9fcf-460b-9154-c775827206eb,Island Oasis Coupon Organizer,"Description Coupon Organizer in a fun colorful fabric -island oasis, Great Size for the ""basic"" couponer - holds up to 500 coupons with ease, and is made long enough so that you…",Consuelo Aguirre,0,"Grocery Items,Grocery Coupons,Care Organization,Coupon Organization,Extreme Couponing,Couponing 101,Life Binder,Save My Money,Love Coupons",image,https://i.pinimg.com/originals/65/bb/ea/65bbeaf458907bb079317d8303c4fa0e.jpg,1,Local save in /data/finance,finance
8304,5b6d0913-25e4-43ab-839d-85d5516f78a4,The #1 Reason You’re Not His Priority Anymore - Matthew Coast,#lovequotes #matchmaker #matchmadeinheaven #loveyourself #respectyourself,Commitment Connection,51k,"Wise Quotes,Quotable Quotes,Words Quotes,Wise Words,Quotes To Live By,Great Quotes,Motivational Quotes,Inspirational Quotes,Funny Quotes",image,https://i.pinimg.com/originals/c6/64/ee/c664ee71524fb5a6e7b7b49233f93b43.png,1,Local save in /data/quotes,quotes
8731,ea760f71-febf-4023-b592-d17396659039,20 Koi Fish Tattoos For Lucky Men,"Koi fish tattoos are a popular choice for men who want to make a statement, thanks to their rich symbolism and bold design.",TheTrendSpotter,211k,"Dr Tattoo,Wörter Tattoos,Pisces Tattoos,Tatoo Art,Dream Tattoos,Dope Tattoos,Mini Tattoos,Finger Tattoos,Body Art Tattoos",image,https://i.pinimg.com/originals/8a/0c/0a/8a0c0a7b6236565c519acd41ad1a52c0.jpg,1,Local save in /data/tattoos,tattoos
1313,44662045-e891-4821-8a19-ebe7eedd371a,Liquid Lash Extensions Mascara,"Instantly create the look of lash extensions with this award-winning, best-selling mascara that won't clump, flake or smudge. Available in 3 shades!",Thrive Causemetics,43k,"N,o, ,T,a,g,s, ,A,v,a,i,l,a,b,l,e",video,https://i.pinimg.com/videos/thumbnails/originals/69/84/e2/6984e20f3e262098fa9c0614c3453254.0000001.jpg,1,Local save in /data/beauty,beauty
4315,21b59ba9-829d-4c33-8c27-4cd4c56d26b8,Podcasts for Teachers or Parents of Teenagers,"Podcasts for Teachers or Parents of Teenagers: Teaching teens middle school and high school can feel joyful and rewarding most days, but can also frustrate you with one challeng…",Math Giraffe,25k,"Middle School Classroom,High School Students,High School Teachers,Middle School Tips,High School Counseling,Ela Classroom,High School Science,Future Classroom,Google Classroom",image,https://i.pinimg.com/originals/50/19/31/501931a27ee4d076658980851b995b2c.jpg,1,Local save in /data/education,education
10794,c4bd2577-a7bb-4409-bb7a-17d5ed7e1cf1,TireBuyer,Nissan GT-R. Sick.,Ray Uyemura,437,"Lowrider,Old Vintage Cars,Antique Cars,Austin Martin,Nissan Gtr Black,Jaguar,1959 Cadillac,Cadillac Ct6,Old School Cars",image,https://i.pinimg.com/originals/0d/29/9f/0d299f3df020395aa7ce8387f40fbeed.jpg,1,Local save in /data/vehicles,vehicles
5494,8fb2af68-543b-4639-8119-de33d28706ed,Dave Ramsey's 7 Baby Steps: What Are They And Will They Work For You,"If you love budgeting, make sure to give Dave Ramsey's 7 Baby Steps a try. Follow these steps to begin your debt snowball, build an emergency fund, invest and reach rich

In [ ]:
"""
Why was this code used?

This code sets up a streaming query using Delta Lake as the output format,
with various configuration options, 
and defines where the output of the streaming query will be written .
    
"""

query = (
    df_stream_pin
    .writeStream
    .format("delta")   
    .option("option_key", "value") 
    .queryName("df_stream_pin")  
    .outputMode("append")  
    .option("checkpointLocation", "/tmp/kinesis/_checkpoints/") \
    .table("0abf7f0cd605_pin_table") 

)

In [ ]:
query = (
    df_stream_geo 
    .writeStream
    .format("delta")      
    .option("option_key", "value")   
    .queryName("df_stream_geo")    
    .outputMode("append")  
    .option("checkpointLocation", "/tmp/kinesis/_checkpoints/") \
    .table("0abf7f0cd605_geo_table") 
    

)

In [ ]:
query = (
    df_stream_user 
    .writeStream
    .format("delta")     
    .option("option_key", "value")  
    .queryName("df_stream_user") 
    .outputMode("append") 
    .option("checkpointLocation", "/tmp/kinesis/_checkpoints/") \
    .table("0abf7f0cd605_user_table") 

)